# FinTech Project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests

## 1. Importing Transaction data

### 1.1  historical 2016 data

In [2]:
df_old = pd.read_csv('/Users/Stephanie_Zhang/Desktop/LendingClub项目/LoanStats3c.csv')

### 1.2 current 2017 data (import from Lending Club API)

In [4]:
import requests
import json
header = { 'Authorization': 'MFlsdMVZe6dtdw/RzMruKtl6y1k='}
r = requests.get("https://api.lendingclub.com/api/investor/v1/loans/listing", headers = header)

In [5]:
type(r.json())

dict

In [6]:
data = r.json()
myData = data['loans']
with open('testdata.txt', 'w') as outfile:
    json.dump(myData, outfile)
df_Curr = pd.read_json('testdata.txt')

In [7]:
len(df_Curr)

155

In [8]:
df_Curr.columns

Index(['accNowDelinq', 'accOpenPast24Mths', 'acceptD', 'addrState', 'addrZip',
       'allUtil', 'annualInc', 'annualIncJoint', 'applicationType',
       'avgCurBal',
       ...
       'totCollAmt', 'totCurBal', 'totHiCredLim', 'totalAcc', 'totalBalExMort',
       'totalBalIl', 'totalBcLimit', 'totalCuTl', 'totalIlHighCreditLimit',
       'totalRevHiLim'],
      dtype='object', length=103)

### 1.3 Column Name Adjustment for the data in 2016 and 2017

In [10]:
listOld = []
for column_name in df_old.columns.values:
    Name = column_name.replace("_","")
    listOld.append(Name)
#     print (Name)

In [11]:
def CapDown (string):
    for i in range (len(string)):
        string = string[0: i] + string[i].lower() + string[i+1:]
    return string

In [12]:
listCurr = []
for column_name in df_Curr.columns.values:
    listCurr.append(CapDown(column_name))
#     print (CapDown(column_name))

### 1.4 Variable Column Compare between the data of 2016 and 2017

In [13]:
def compareCol (list1, list2):
    same = []
    different = []
    for i in range (len(list1)):
        if list1[i] in list2:
            same.append(list1[i])
        else:
            different.append(list1[i])
    return same, different

In [14]:
sameCurr, differentCurr = compareCol(listCurr, listOld)
sameOld, differentOld = compareCol(listOld, listCurr)
print (differentCurr)
print (differentOld)

['acceptd', 'addrzip', 'creditpulld', 'desc', 'expd', 'expdefaultrate', 'ficorangehigh', 'ficorangelow', 'fundedamount', 'id', 'ilsexpd', 'investorcount', 'isincv', 'isincvjoint', 'listd', 'loanamount', 'memberid', 'numacctsever120ppd', 'reviewstatus', 'reviewstatusd', 'servicefeerate']
['loanamnt', 'fundedamnt', 'fundedamntinv', 'verificationstatus', 'issued', 'loanstatus', 'pymntplan', 'title', 'zipcode', 'outprncp', 'outprncpinv', 'totalpymnt', 'totalpymntinv', 'totalrecprncp', 'totalrecint', 'totalreclatefee', 'recoveries', 'collectionrecoveryfee', 'lastpymntd', 'lastpymntamnt', 'nextpymntd', 'lastcreditpulld', 'policycode', 'verificationstatusjoint', 'numacctsever120pd']


In [15]:
def LCS (str1, str2):
    m = len(str1)
    n = len(str2)
    max = 0
    common = list()
    count = [[0] * (n + 1) for x in range (m + 1)]
    for i in range (m + 1):
        count [i][0] = 0
    for j in range (n + 1):
        count [0][j] = 0
    for i in range (1, m + 1):
        for j in range (1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                count[i][j] = count[i - 1][j - 1] + 1
                if max < count[i][j]:
                    max = count[i][j]
                    common.append(str1[i - 1])
            else:
                count[i][j] = 0
    return (max)

In [16]:
print (LCS ('happy', 'halloppy'))

3


In [17]:
for i in range (len(differentCurr)):
    for j in range (len(differentOld)):
        if LCS (differentCurr[i], differentOld[j]) > 3 : 
            print (LCS (differentCurr[i], differentOld[j]), differentCurr[i], differentOld[j])

11 creditpulld lastcreditpulld
8 fundedamount fundedamnt
8 fundedamount fundedamntinv
5 isincvjoint verificationstatusjoint
6 loanamount loanamnt
4 loanamount loanstatus
16 numacctsever120ppd numacctsever120pd
6 reviewstatus verificationstatus
6 reviewstatus loanstatus
6 reviewstatus verificationstatusjoint
6 reviewstatusd verificationstatus
6 reviewstatusd loanstatus
6 reviewstatusd verificationstatusjoint
4 servicefeerate totalreclatefee


### 1.5 Adjust the Columns

In [18]:
# rename according to the name comparison
df_Curr.columns = listCurr
df_old.columns = listOld
# df_Curr.columns, df_old.columns

df_Curr = df_Curr.rename(columns={'creditpulld': 'lastcreditpulld'})
df_Curr = df_Curr.rename(columns={'fundedamount': 'fundedamnt'})
df_Curr = df_Curr.rename(columns={'loanamount': 'loanamnt'})
df_Curr = df_Curr.rename(columns={'numacctsever120ppd': 'numacctsever120pd'})

In [19]:
# another round of calculating the same and difference list
sameCurr2, differentCurr2 = compareCol(df_Curr.columns, df_old.columns)
sameOld2, differentOld2 = compareCol(df_old.columns, df_Curr.columns)
print (differentCurr2)
print (differentOld2)

['acceptd', 'addrzip', 'desc', 'expd', 'expdefaultrate', 'ficorangehigh', 'ficorangelow', 'id', 'ilsexpd', 'investorcount', 'isincv', 'isincvjoint', 'listd', 'memberid', 'reviewstatus', 'reviewstatusd', 'servicefeerate']
['fundedamntinv', 'verificationstatus', 'issued', 'loanstatus', 'pymntplan', 'title', 'zipcode', 'outprncp', 'outprncpinv', 'totalpymnt', 'totalpymntinv', 'totalrecprncp', 'totalrecint', 'totalreclatefee', 'recoveries', 'collectionrecoveryfee', 'lastpymntd', 'lastpymntamnt', 'nextpymntd', 'policycode', 'verificationstatusjoint']


In [20]:
# will delete the difference in new list differentCurr2
for i in range(len(differentCurr2)):
    df_Curr = df_Curr.drop(differentCurr2[i], 1)
len(df_Curr.columns)

86

In [21]:
# will delete the difference in new list differentOld2:
for i in range(len(differentOld2)):
    if differentOld2[i] != 'loanstatus' and differentOld2[i] != 'issued':
        df_old = df_old.drop(differentOld2[i], 1)
len(df_old.columns)

88

# 2. Split train & test data, and aggregate into Full_data

### 2.1 Target variable: loanstatus

In [22]:
print(df_old.loanstatus.value_counts())
df_old = df_old.query("loanstatus == 'Fully Paid' or loanstatus == 'Charged Off'")
df_old['loanstatus'] = df_old.loanstatus.map({"Charged Off": 0, "Fully Paid": 1})
# print(df_old.loanstatus)
print(df_old.loanstatus.value_counts())

Fully Paid            114533
Current                83597
Charged Off            32263
Late (31-120 days)      2737
In Grace Period         1845
Late (16-30 days)        647
Default                    7
Name: loanstatus, dtype: int64
1    114533
0     32263
Name: loanstatus, dtype: int64


### 2.2 Split into test and train data according to timestamp

In [23]:
df_old['issued'] = df_old.issued.apply(lambda x: x.split("-")[0])
df_test = df_old.query("issued =='Dec' or issued == 'Nov' or issued == 'Oct'")
df_train = df_old.query("issued !='Dec' and issued != 'Nov' and issued != 'Oct'")
df_test['flag'] = 0
df_train['flag'] = 1
df_Curr['flag'] = 2
Full_data = pd.concat((df_test, df_train, df_Curr), axis=0, ignore_index=True)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
print(len(Full_data.columns), len(Full_data))
print(len(df_Curr.columns), len(df_old.columns), len(listCurr), len(listOld), len(df_Curr), len(df_old))

89 146951
87 88 103 107 155 146796


# Grouping variables

In [26]:
for column_name in Full_data.columns.values:
    print (column_name, ":  ", Full_data[column_name].dtype)

accnowdelinq :   int64
accopenpast24mths :   int64
addrstate :   object
allutil :   float64
annualinc :   float64
annualincjoint :   float64
applicationtype :   object
avgcurbal :   float64
bcopentobuy :   float64
bcutil :   float64
chargeoffwithin12mths :   int64
collections12mthsexmed :   int64
delinq2yrs :   int64
delinqamnt :   int64
dti :   float64
dtijoint :   float64
earliestcrline :   object
emplength :   object
emptitle :   object
flag :   int64
fundedamnt :   int64
grade :   object
homeownership :   object
ilutil :   float64
initialliststatus :   object
inqfi :   float64
inqlast12m :   float64
inqlast6mths :   int64
installment :   float64
intrate :   object
issued :   object
lastcreditpulld :   object
loanamnt :   int64
loanstatus :   float64
maxbalbc :   float64
mortacc :   int64
mosinoldilacct :   float64
mosinoldrevtlop :   int64
mosinrcntrevtlop :   int64
mosinrcnttl :   int64
mthssincelastdelinq :   float64
mthssincelastmajorderog :   float64
mthssincelastrecord :   flo

## 3. Checking with Null

In [27]:
Full_data.isnull().sum() 

accnowdelinq                   0
accopenpast24mths              0
addrstate                      0
allutil                   146796
annualinc                      0
annualincjoint            146947
applicationtype                0
avgcurbal                      6
bcopentobuy                 1497
bcutil                      1608
chargeoffwithin12mths          0
collections12mthsexmed         0
delinq2yrs                     0
delinqamnt                     0
dti                            0
dtijoint                  146947
earliestcrline                 0
emplength                     13
emptitle                    8238
flag                           0
fundedamnt                     0
grade                          0
homeownership                  0
ilutil                    146821
initialliststatus              0
inqfi                     146796
inqlast12m                146796
inqlast6mths                   0
installment                    0
intrate                        0
          

#### drop null columns

In [28]:
# drop null columns
Full_data = Full_data.drop('openacc6m', 1)
Full_data = Full_data.drop('openil12m', 1)
Full_data = Full_data.drop('openil24m', 1)
Full_data = Full_data.drop('openil6m', 1)
Full_data = Full_data.drop('openrv12m', 1)
Full_data = Full_data.drop('openrv24m', 1)
Full_data = Full_data.drop('allutil', 1)
Full_data = Full_data.drop('inqfi', 1)
Full_data = Full_data.drop('inqlast12m', 1)
Full_data = Full_data.drop('totalbalil', 1)
Full_data = Full_data.drop('totalcutl', 1)

In [29]:
# check the Full_data again
# np.sum(Full_data.isnull())

## 4. Feature Engineering

### 4.1 Grouping variables into continuous and categorical

In [30]:
# Value grouping into continuous and categorical
Con_cols = [i for i in Full_data.columns if (Full_data.dtypes[i] =='float64' or Full_data.dtypes[i] == 'int64')]
Cat_cols = [i for i in Full_data.columns if (i not in Con_cols) & (i !='id')]
len(Con_cols), len(Cat_cols)
print(Con_cols)
print(Cat_cols)

['accnowdelinq', 'accopenpast24mths', 'annualinc', 'annualincjoint', 'avgcurbal', 'bcopentobuy', 'bcutil', 'chargeoffwithin12mths', 'collections12mthsexmed', 'delinq2yrs', 'delinqamnt', 'dti', 'dtijoint', 'flag', 'fundedamnt', 'ilutil', 'inqlast6mths', 'installment', 'loanamnt', 'loanstatus', 'maxbalbc', 'mortacc', 'mosinoldilacct', 'mosinoldrevtlop', 'mosinrcntrevtlop', 'mosinrcnttl', 'mthssincelastdelinq', 'mthssincelastmajorderog', 'mthssincelastrecord', 'mthssincercntil', 'mthssincerecentbc', 'mthssincerecentbcdlq', 'mthssincerecentinq', 'mthssincerecentrevoldelinq', 'numacctsever120pd', 'numactvbctl', 'numactvrevtl', 'numbcsats', 'numbctl', 'numiltl', 'numoprevtl', 'numrevaccts', 'numrevtlbalgt0', 'numsats', 'numtl120dpd2m', 'numtl30dpd', 'numtl90gdpd24m', 'numtloppast12m', 'openacc', 'pcttlnvrdlq', 'percentbcgt75', 'pubrec', 'pubrecbankruptcies', 'revolbal', 'taxliens', 'totalacc', 'totalbalexmort', 'totalbclimit', 'totalilhighcreditlimit', 'totalrevhilim', 'totcollamt', 'totcurb

### 4.2 target variable distribution

In [31]:
Full_data.loanstatus.value_counts()

1.0    114533
0.0     32263
Name: loanstatus, dtype: int64

In [33]:
print(len(Full_data.columns), len(Full_data))
print(len(df_Curr.columns), len(df_old.columns), len(listCurr), len(listOld), len(df_Curr), len(df_old))

78 146951
87 88 103 107 155 146796


### 4.3 Feature engineering

#### 4.3.1 Grade, subgrade

In [34]:
# grade
grade_dic = {"A": 1, "B": 2, "C": 3, "D": 4, "E":5, "F":6, "G":7} 
Full_data.grade = Full_data.grade.map(grade_dic)
# subgrade
Full_data.subgrade = Full_data.subgrade.apply(lambda x: (grade_dic[x[0]]) + int(x[1])/10)

#### 4.3.2 Addrstate

In [36]:
# addrstate
addrstate_freq = Full_data.groupby("addrstate").size().reset_index()
addrstate_freq.columns = ["addrstate", "addrstate_freq"]
Full_data = pd.merge(Full_data, addrstate_freq, how = "left", on = "addrstate")
# drop addrstate
Full_data.drop('addrstate', axis =1)

#### 4.3.3 emptitle

In [38]:
#emptitle
emptitle_freq = Full_data.groupby("emptitle").size().reset_index()
emptitle_freq.columns = ["emptitle", "emptitle_freq"]
Full_data = pd.merge(Full_data, emptitle_freq, how = "left", on = "emptitle")

#### 4.3.4 null -> 'n/a'

In [40]:
# all data
Full_data.replace('n/a', np.nan, inplace = True)

#### 4.3.5 intrate

In [41]:
# intrate
Full_data.intrate = pd.Series(Full_data.intrate).str.replace('%', '').astype(float)

#### 4.3.6 revoluntil

In [42]:
# revoluntil
Full_data.revolutil = pd.Series(Full_data.revolutil).str.replace('%', '').astype(float)

#### 4.3.7 emplength (drop)

In [43]:
# Full_data.emplength ---deleted, for current data is not consistant with the historical data，
# which was supposed to be 1-10 years
Full_data = Full_data.drop('emplength', 1)

#### 4.3.8 categorical variables - one-hot encoding

In [44]:
dummy = ["applicationtype", "homeownership","initialliststatus", "purpose","term"]

In [ ]:
from sklearn import preprocessing
# lets try if label encoder can replace np.nan
# full_data = full_data.replace(np.nan, 'null')

for col in dummy:
    print ("Label encoding  %s" % (col))
    LBL = preprocessing.LabelEncoder() #1
    LBL.fit(Full_data[col])#2
    labels = dict(zip(Full_data[col].unique()
               , LBL.transform(Full_data[col].unique())))
    print (labels)
    print (LBL.classes_)
    Full_data[col]=LBL.transform(Full_data[col])#3

In [49]:
# 'applicationtype','homeownership','initialliststatus','purpose'
one_hot = pd.get_dummies(Full_data['homeownership'])
Full_data = Full_data.drop('homeownership', axis =1)
Full_data = Full_data.join(one_hot)

In [52]:
Full_data.columns = Full_data.columns.astype(str)

In [53]:
#  rename 
Full_data = Full_data.rename(columns={'0': 'homeownership_0'})
Full_data = Full_data.rename(columns={'1': 'homeownership_1'})
Full_data = Full_data.rename(columns={'2': 'homeownership_2'})
Full_data = Full_data.rename(columns={'3': 'homeownership_3'})
# Full_data.columns

In [54]:
one_hot1 = pd.get_dummies(Full_data['initialliststatus'])
Full_data = Full_data.drop('initialliststatus', axis =1)
Full_data = Full_data.join(one_hot1)

In [55]:
Full_data.columns = Full_data.columns.astype(str)
Full_data = Full_data.rename(columns={'0': 'initialliststatus_0'})
Full_data = Full_data.rename(columns={'1': 'initialliststatus_1'})
Full_data = Full_data.rename(columns={'2': 'initialliststatus_2'})

In [57]:
one_hot3 = pd.get_dummies(Full_data['purpose'])
Full_data = Full_data.drop('purpose', axis =1)
Full_data = Full_data.join(one_hot3)

In [58]:
Full_data.columns = Full_data.columns.astype(str)
Full_data = Full_data.rename(columns={'0': 'purpuse_0'})
Full_data = Full_data.rename(columns={'1': 'purpuse_1'})
Full_data = Full_data.rename(columns={'2': 'purpuse_2'})
Full_data = Full_data.rename(columns={'3': 'purpuse_3'})
Full_data = Full_data.rename(columns={'4': 'purpuse_4'})
Full_data = Full_data.rename(columns={'5': 'purpuse_5'})
Full_data = Full_data.rename(columns={'6': 'purpuse_6'})
Full_data = Full_data.rename(columns={'7': 'purpuse_7'})
Full_data = Full_data.rename(columns={'8': 'purpuse_8'})
Full_data = Full_data.rename(columns={'9': 'purpuse_9'})
Full_data = Full_data.rename(columns={'10': 'purpuse_10'})
Full_data = Full_data.rename(columns={'11': 'purpuse_11'})
Full_data = Full_data.rename(columns={'12': 'purpuse_12'})

In [60]:
one_hot4 = pd.get_dummies(Full_data['applicationtype'])
Full_data = Full_data.drop('applicationtype', axis =1)
Full_data = Full_data.join(one_hot4)

In [61]:
Full_data = Full_data.rename(columns={'0': 'applicationtype_0'})
Full_data = Full_data.rename(columns={'1': 'applicationtype_1'})

#### 4.3.9 Drop Items

In [62]:
# i dropped three columns that there is little to deal with for now.
Full_data = Full_data.drop('term', axis = 1)
Full_data = Full_data.drop('earliestcrline', axis = 1)
Full_data = Full_data.drop('lastcreditpulld', axis = 1)
Full_data = Full_data.drop('addrstate', axis = 1) #BEFORE SHOULD HAVE DELETED
Full_data = Full_data.drop('emptitle', axis = 1) #BEFORE SHOULD HAVE DELETED
Full_data = Full_data.drop('issued', axis = 1) #BEFORE SHOULD HAVE DELETED
# drop the nan
Full_data = Full_data.drop('annualincjoint', axis = 1)
Full_data = Full_data.drop('dtijoint', axis = 1)
Full_data = Full_data.drop('ilutil', axis = 1)
Full_data = Full_data.drop('maxbalbc', axis = 1)
Full_data = Full_data.drop('mthssincercntil', axis = 1)

#### 4.3.10 Fill NA with median

In [64]:
Full_data = Full_data.fillna(Full_data.median())

# 5. Train & Test Split

In [65]:
train = Full_data.loc[Full_data['flag'] == 0]
test = Full_data.loc[Full_data['flag'] == 1]
test_predict = Full_data.loc[Full_data['flag'] == 2]

In [66]:
print (len(train), len(test), len(test_predict))
print (len(Full_data))

37577 109219 155
146951


In [67]:
X_train = train.drop("loanstatus", axis=1)
Y_train = train["loanstatus"]
X_test = test.drop("loanstatus", axis=1)
Y_test = test["loanstatus"]
X_test_predict = test_predict.drop("loanstatus", axis=1)

In [68]:
print(len(X_train), len(X_train.columns), len(Y_train))

37577 85 37577


#### Result:   train set: 37577,   test set: 109219,   predict: 155 ,  85 variables

# 6. Model Selection & Tuning

In [69]:
Full_data.head(4)

,accnowdelinq,accopenpast24mths,annualinc,avgcurbal,bcopentobuy,bcutil,chargeoffwithin12mths,collections12mthsexmed,delinq2yrs,delinqamnt,...,purpuse_5,purpuse_6,purpuse_7,purpuse_8,purpuse_9,purpuse_10,purpuse_11,purpuse_12,0,1
0,0,7,58000.0,9536.0,7599.0,41.5,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,5,78000.0,29828.0,9525.0,4.7,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,4,63800.0,4232.0,324.0,97.8,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,6,50000.0,5857.0,332.0,93.2,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [70]:
# # Check Null
# Full_data.iloc[:,1:40].isnull().sum() 
# Full_data.iloc[:,41:90].isnull().sum() 

### 6.1 Random Forest Classifier

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
RF = RandomForestClassifier()#linearregression
RF = RF.fit(X_train, Y_train)
y_pred = RF.predict(X_test)

In [72]:
print (metrics.classification_report(Y_test, y_pred))# accuracy

             precision    recall  f1-score   support

        0.0       0.35      0.27      0.30     22873
        1.0       0.82      0.87      0.84     86346

avg / total       0.72      0.74      0.73    109219



### 6.2 Decision Tree Classifier

In [73]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=5)  #(可以改max depth)
dtc.fit(X_train,Y_train)
y_pred1 = dtc.predict(X_test)

In [76]:
print (metrics.classification_report(Y_test, y_pred1))# accuracy

             precision    recall  f1-score   support

        0.0       0.44      0.11      0.18     22873
        1.0       0.80      0.96      0.88     86346

avg / total       0.73      0.78      0.73    109219



### 6.3 Parameter tuning: Grid Search

In [77]:
from sklearn import metrics, grid_search
from sklearn.model_selection import cross_val_score
def search_model(x_train, y_train, est, param_grid, n_jobs, cv, refit=False):
##Grid Search for the best model
    model = grid_search.GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring    = 'f1_weighted',
                                     verbose    = 10,
                                     n_jobs  = n_jobs,
                                     iid    = True,
                                     refit    = refit,
                                     cv      = cv)
    # Fit Grid Search Model
    model.fit(x_train, y_train)
#     print("Best score: %0.3f" % model.best_score_)
#     print("Best parameters set:", model.best_params_)
#     print("Scores:", model.grid_scores_)    
    return model

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
np.random.seed(1234)
param_grid = {'n_estimators':[100,300,500],
             'criterion':['gini', 'entropy']}

RF = search_model(X_train.values
            , Y_train.values
            , RandomForestClassifier()
            , param_grid
            , -1
            , 5)

### 6.4 Voting ensemble

In [78]:
# voting ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
clf1 = RandomForestClassifier(n_estimators = 500, random_state = 1234, criterion = 'entropy')
# results from your gridsearch
clf2 = KNeighborsClassifier(n_neighbors=3)
# results from your gridsearch
eclf = VotingClassifier(estimators=[('Random_Forest',clf1), ('KNN', clf2)], voting='hard')
for clf, label in zip([clf1, clf2, eclf], ['Random_Forest', 'KNN', 'Ensemble']):
    scores = cross_val_score(clf, X_train, Y_train, cv=5, scoring='f1_weighted')
    print ("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.70 (+/- 0.01) [Random_Forest]
Accuracy: 0.67 (+/- 0.00) [KNN]
Accuracy: 0.69 (+/- 0.00) [Ensemble]


In [80]:
eclf.fit(X_train, Y_train)

VotingClassifier(estimators=[('Random_Forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
...owski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform'))],
         n_jobs=1, voting='hard', weights=None)